In [1]:
import os
from glob import glob
from tqdm import tqdm
import datetime
import hail as hl

from hail.plot import show
import pandas as pd
from pprint import pprint
hl.plot.output_notebook()

from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

Loading BokehJS ...

Loading BokehJS ...

#### Start an [Apache Spark](https://en.wikipedia.org/wiki/Apache_Spark) instance

In [2]:
log_file_name = f"logs/hail-{datetime.datetime.now():%Y-%m-%d-%H-%M-%S}.log"
# run spark
spark_conf = SparkConf().setAppName("hail-test")
# .setMaster("spark://spark-master:7077")
spark_conf.set("spark.hadoop.fs.s3a.endpoint", "http://lifemap-minio:9000/")
spark_conf.set("spark.hadoop.fs.s3a.access.key", "root")
spark_conf.set("spark.hadoop.fs.s3a.secret.key", "passpass" )
spark_conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
spark_conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
spark_conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark_conf.set("spark.hadoop.fs.s3a.connection.maximum", 1024);
spark_conf.set("spark.hadoop.fs.s3a.threads.max", 1024);
spark_conf.set("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

try:
    sc = SparkContext(conf=spark_conf)
except:
    print ("Spark session already up")

#### Create bucket on [Minio](https://min.io/) if it does not exists

In [3]:
import boto3
from botocore.exceptions import NoCredentialsError

# S3 configuration
s3 = boto3.client(
    's3',
    endpoint_url="http://lifemap-minio:9000",
    aws_access_key_id="root",
    aws_secret_access_key="passpass",
)

bucket_name = "data-hail"

# Check if the bucket exists, if not, create it
try:
    s3.head_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' exists.")
except Exception:
    # If the bucket does not exist, create it
    s3.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' created.")

Bucket 'data-hail' exists.


### [Hail](https://hail.is/) initialization

In [ ]:
hl.init(sc=sc, log=log_file_name)

#### Set filenames

In [5]:
## VCF
vcf_fn = 'data/1kg.vcf'
## Annotation file
annotations_fn = 'data/1kg_annotations.txt'
## Matrix table
mt_fn = 's3://data-hail/1kg.mt'

print (f"VCF fn: {vcf_fn}")
print (f"Annotation file fn: {annotations_fn}")
print (f"Matrix table fn: {mt_fn}")

VCF fn: data/1kg.vcf
Annotation file fn: data/1kg_annotations.txt
Matrix table fn: s3://data-hail/1kg.mt


#### Reading vcf with Pandas (N/A if the vcf is stored on s3)

In [6]:
vcf_pd = None
if "s3://" not in vcf_fn: 
    vcf_pd = pd.read_csv(vcf_fn, sep="\t", header=109, low_memory=False)
vcf_pd

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG00096,...,NA20877,NA20888,NA20910,NA21101,NA21113,NA21114,NA21116,NA21118,NA21133,NA21143
0,1,904165,.,G,A,52346.37,.,AC=518;AF=1.03000e-01;AN=5020;BaseQRankSum=-3....,GT:AD:DP:GQ:PL,"0/0:4,0:4:12:0,12,147",...,"0/0:3,0:3:9:0,9,111","0/0:11,0:11:32:0,32,349","0/0:5,0:5:15:0,15,173","0/0:4,0:4:12:0,12,161","0/0:5,0:5:15:0,15,189","0/0:10,0:10:30:0,30,339","0/0:7,0:7:21:0,21,286","0/0:13,0:13:39:0,39,498","0/0:10,0:10:30:0,30,366","0/0:10,0:10:30:0,30,361"
1,1,909917,.,G,A,1576.94,.,AC=18;AF=3.72700e-03;AN=4830;BaseQRankSum=-1.4...,GT:AD:DP:GQ:PL,"0/0:4,0:4:12:0,12,160",...,"0/0:4,0:4:12:0,12,146","0/0:10,0:10:30:0,30,357","0/0:10,0:10:29:0,29,312",./.:.:.:.:.,"0/0:4,0:4:12:0,12,154","0/0:14,0:14:42:0,42,499","0/0:10,0:10:30:0,30,410","0/0:12,0:12:36:0,36,451","0/0:7,0:7:21:0,21,223","0/0:6,0:6:17:0,17,191"
2,1,986963,.,C,T,398.06,.,AC=5;AF=1.09000e-03;AN=4588;BaseQRankSum=1.253...,GT:AD:DP:GQ:PL,"0/0:3,0:3:9:0,9,98",...,"0/0:2,0:2:6:0,6,71","0/0:11,0:11:33:0,33,332","0/0:8,0:8:28:0,28,271","0/0:2,0:2:5:0,5,43",./.:.:.:.:.,"0/0:8,0:8:26:0,26,252",./.:.:.:.:.,"0/0:5,0:5:15:0,15,176","0/0:6,0:6:18:0,18,209","0/0:8,0:8:29:0,29,268"
3,1,1563691,.,T,G,1090.75,.,AC=64;AF=1.30000e-02;AN=4766;BaseQRankSum=-3.8...,GT:AD:DP:GQ:PL,./.:.:.:.:.,...,"0/0:4,0:4:12:0,12,124","0/0:7,0:7:20:0,20,184","0/0:2,0:2:6:0,6,76","0/0:2,0:2:6:0,6,73",./.:.:.:.:.,"0/1:6,4:10:99:110,0,140","0/0:7,0:7:21:0,21,282","0/0:11,0:11:33:0,33,404","0/0:4,2:6:6:0,6,115","0/0:16,0:16:44:0,44,481"
4,1,1707740,.,T,G,93517.82,.,AC=997;AF=1.98000e-01;AN=5034;BaseQRankSum=-4....,GT:AD:DP:GQ:PL,"0/1:2,3:5:67:82,0,67",...,"0/1:1,3:4:26:94,0,26","0/0:10,0:10:30:0,30,356","0/1:5,3:8:65:65,0,147","0/0:1,0:1:3:0,3,37","0/0:16,0:16:48:0,48,605","0/0:4,0:4:12:0,12,154","0/1:7,10:17:99:296,0,233","0/1:4,3:7:70:70,0,135","0/1:1,10:11:8:273,0,8","0/0:7,0:7:21:0,21,268"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10874,X,152660491,.,C,G,130357.76,.,AC=1922;AF=3.99000e-01;AN=4822;BaseQRankSum=-4...,GT:AD:DP:GQ:PL,./.:.:.:.:.,...,"1/1:0,1:1:3:39,3,0","0/0:7,0:7:21:0,21,260","0/1:3,2:5:51:51,0,87","0/0:7,0:7:21:0,21,276",./.:.:.:.:.,"0/0:1,0:1:3:0,3,36","1/1:0,6:6:18:199,18,0","0/0:3,0:3:9:0,9,121","1/1:0,3:3:9:109,9,0","0/0:13,0:13:39:0,39,478"
10875,X,153031688,.,C,T,2172.66,.,AC=45;AF=9.65300e-03;AN=4662;BaseQRankSum=2.13...,GT:AD:DP:GQ:PL,"0/0:1,0:1:3:0,3,26",...,"0/0:1,0:1:3:0,3,40","0/0:12,0:12:36:0,36,394","0/0:13,0:13:39:0,39,433","0/0:3,0:3:9:0,9,90",./.:.:.:.:.,"0/0:11,0:11:32:0,32,296","0/0:4,0:4:12:0,12,154","0/0:3,0:3:9:0,9,100","0/0:7,0:7:21:0,21,237","0/0:8,0:8:21:0,21,184"
10876,X,153674876,.,C,T,627.57,.,AC=7;AF=1.45200e-03;AN=4822;BaseQRankSum=-9.88...,GT:AD:DP:GQ:PL,"0/0:2,0:2:6:0,6,63",...,"0/0:2,0:2:6:0,6,66","0/0:8,0:8:24:0,24,241","0/0:4,0:4:12:0,12,149","0/0:3,0:3:9:0,9,88","0/0:4,0:4:12:0,12,125","0/0:10,0:10:29:0,29,273","0/0:2,0:2:6:0,6,78","0/0:7,0:7:21:0,21,258","0/0:4,0:4:12:0,12,128","0/0:8,0:8:24:0,24,254"
10877,X,153706320,.,C,T,929.98,.,AC=13;AF=2.68200e-03;AN=4848;BaseQRankSum=3.64...,GT:AD:DP:GQ:PL,./.:.:.:.:.,...,"0/0:3,0:3:9:0,9,99","0/0:7,0:7:21:0,21,221","0/0:5,0:5:15:0,15,169","0/0:2,0:2:6:0,6,61","0/0:8,0:8:24:0,24,255","0/0:4,0:4:12:0,12,108","0/0:5,0:5:15:0,15,172","0/0:2,0:2:6:0,6,70","0/0:3,0:3:9:0,9,95","0/0:8,0:8:24:0,24,245"


### Import VCF to hail as Matrix Table

The VCF file must be converted into the [Hail Matrix Table](https://hail.is/docs/0.2/overview/matrix_table.html) data structure.
This structure builds upon the [Hail Table](https://hail.is/docs/0.2/hail.Table.html#hail.Table) and comprises four key components:

- a **two-dimensional matrix of entry fields** where each entry **is indexed by row key(s) and column key(s)**
- a corresponding **rows table** that stores all of the row fields that are constant for every column in the dataset
- a corresponding **columns table** that stores all of the column fields that are constant for every row in the dataset
- a set of **global fields** that are constant for every entry in the dataset

Every Hail Table has a **key** that controls both the order of the rows in the table and the ability to join or annotate one table with the information in another table. 
Matrix tables have **two keys**: a **row key** and a **column key**. Row fields are indexed by the row key, column fields are indexed by the column key, and entry fields are indexed by the row key and the column key.

In [ ]:
## Read a vcf file, convert and write it as matrix table
_ = hl.import_vcf(vcf_fn).write(mt_fn, overwrite=True) # assign this to a dummy variable to avoid errors

In [8]:
## Read the matrix table from the file and assign it to the mt vaiable
mt = hl.read_matrix_table(mt_fn)
row_table = mt.rows()
col_table = mt.cols()
entry_fields = mt.entries()

print (f"Row table: {row_table}")
print (f"Col table: {col_table}")
print (f"Entries: {entry_fields}")

Row table: <hail.table.Table object at 0x7f79d475c340>
Col table: <hail.table.Table object at 0x7f79d4768880>
Entries: <hail.table.Table object at 0x7f79d475f2b0>


![alt text](immagini/vcf_matrix_table.png "Title")

In [9]:
## Summary of the matrix table:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        set: str
    }
----------------------------------------
Entry fields:
    'GT': call
    'AD': array<int32>
    'DP': int32
    'GQ': int32
    'PL': array<int32>
----------------------------------------
Colu

#### Row table:

In [10]:
row_table.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        set: str
    } 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [11]:
row_table.show()

+---------------+------------+------+----------+----------+--------------+
| locus         | alleles    | rsid |     qual | filters  | info.AC      |
+---------------+------------+------+----------+----------+--------------+
| locus<GRCh37> | array<str> | str  |  float64 | set<str> | array<int32> |
+---------------+------------+------+----------+----------+--------------+
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:909917      | ["G","A"]  | NA   | 1.58e+03 | NA       | [18]         |
| 1:986963      | ["C","T"]  | NA   | 3.98e+02 | NA       | [5]          |
| 1:1563691     | ["T","G"]  | NA   | 1.09e+03 | NA       | [64]         |
| 1:1707740     | ["T","G"]  | NA   | 9.35e+04 | NA       | [997]        |
| 1:2252970     | ["C","T"]  | NA   | 7.36e+02 | NA       | [6]          |
| 1:2284195     | ["T","C"]  | NA   | 1.42e+05 | NA       | [1559]       |
| 1:2779043     | ["T","C"]  | NA   | 2.89e+05 | NA       | [3532]       |
| 1:2944527     | ["G","A"]  | NA   | 1.24e+05 | NA       | [1206]       |
| 1:3761547     | ["C","A"]  | NA   | 1.61e+03 | NA       | [30]         |
+---------------+------------+------+----------+----------+--------------+

+----------------+---------+-------------------+----------------------+
| info.AF        | info.AN | info.BaseQRankSum | info.ClippingRankSum |
+----------------+---------+-------------------+----------------------+
| array<float64> |   int32 |           float64 |              float64 |
+----------------+---------+-------------------+----------------------+
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [3.73e-03]     |    4830 |         -1.48e+00 |             1.26e-01 |
| [1.09e-03]     |    4588 |          1.25e+00 |            -3.77e+00 |
| [1.30e-02]     |    4766 |         -3.87e+01 |            -5.39e+00 |
| [1.98e-01]     |    5034 |         -4.04e+01 |            -2.87e-01 |
| [1.28e-03]     |    4682 |         -1.22e+00 |             1.79e+00 |
| [3.12e-01]     |    4990 |         -4.60e+01 |             3.50e-01 |
| [7.26e-01]     |    4866 |          1.74e+01 |             2.13e+00 |
| [2.45e-01]     |    4928 |          6.30e-02 |            -6.55e-01 |
| [5.95e-03]     |    5044 |         -4.47e+00 |            -8.82e+00 |
+----------------+---------+-------------------+----------------------+

+---------+---------+----------+---------------------+----------------------+
| info.DP | info.DS |  info.FS | info.HaplotypeScore | info.InbreedingCoeff |
+---------+---------+----------+---------------------+----------------------+
|   int32 |    bool |  float64 |             float64 |              float64 |
+---------+---------+----------+---------------------+----------------------+
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   14671 |   False | 5.52e+00 |                  NA |            -5.00e-04 |
|   12398 |   False | 8.34e-01 |                  NA |             1.26e-02 |
|   15357 |   False | 1.90e+03 |                  NA |             2.70e-02 |
|   19902 |   False | 3.31e+00 |                  NA |             3.87e-02 |
|   14900 |   False | 2.82e+00 |                  NA |            -8.20e-03 |
|   18176 |   False | 2.95e+00 |                  NA |             9.25e-02 |
|   12878 |   False | 2.55e+01 |                  NA |             1.44e-01 |
|   17698 |   False | 4.49e-01 |                  NA |             1.23e-01 |
|   16845 |   False | 2.06e+00 |                  NA |            -4.70e-03 |
+---------+---------+----------+---------------------+----------------------+

+--------------+----------------+----------+----------+----------------+
| info.MLEAC   | info.MLEAF     |  info.MQ | info.MQ0 | info.MQRankSum |
+--------------+----------------+----------+----------+----------------+
| array<int32> | array<float64> |  float64 |    int32 |        float64 |
+--------------+----------------+----------+----------+----------------+
| [514]        | [1.02e-01]     

#### Column table 

In [12]:
col_table.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
----------------------------------------
Key: ['s']
----------------------------------------


In [13]:
col_table.show()

""
s
str
"""HG00096"""
"""HG00099"""
"""HG00105"""
"""HG00118"""
"""HG00129"""
"""HG00148"""
"""HG00177"""
"""HG00182"""


#### Counting samples and variants

In [14]:
## Counts of samples and variants
print (" Printing row and column fields with different methods")
# count row fields
print(mt.count_rows())
# same as
print (row_table.count())

# count column fields
print(mt.count_cols())
# same as
print(col_table.count())

n_variants = mt.count_rows() # This can be done accessing directy the row table with mt.rows().count()
n_samples = mt.count_cols()  # This can be done accessing directy the column table with mt.cols().count()

print (f"\n\nTable has {n_variants} variants and {n_samples} samples") 

 Printing row and column fields with different methods
10879
10879
284
284


Table has 10879 variants and 284 samples


#### Entry fields.

In [15]:
entry_fields.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        set: str
    } 
    's': str 
    'GT': call 
    'AD': array<int32> 
    'DP': int32 
    'GQ': int32 
    'PL': array<int32> 
----------------------------------------
Key: ['locus', 'alleles', 's']
----------------------------------------


In [16]:
entry_fields.show(n_samples + 1)

+---------------+------------+------+----------+----------+--------------+
| locus         | alleles    | rsid |     qual | filters  | info.AC      |
+---------------+------------+------+----------+----------+--------------+
| locus<GRCh37> | array<str> | str  |  float64 | set<str> | array<int32> |
+---------------+------------+------+----------+----------+--------------+
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:904165      | ["G","A

#### Global values.
Common values of the matrix table

In [17]:
mt.globals_table().show()

""
""


### Summary:  
In the Hail Matrix Table, VCF data are represented as follows:  

- **Row fields:** Each row corresponds to a variant, and row fields contain information common to that variant (e.g., alleles or quality).  
- **Column fields:** Each column represents a sample, with column fields specifying sample-related attributes such as the sample ID.  
- **Entry fields:** These contain data specific to a (variant, sample) pair, such as the genotype read for a particular sample.  
- **Column key:** The column field used for join operations.  
- **Row key:** The row field used for join operations.  

#### Showing rows, cols and entry field data

We can use the Hail table[`select`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.select) method to pull out 5 variants. 
The select method takes either a string refering to a field name in the table, or a [Hail Expression](https://hail.is/docs/0.2/overview/expressions.html). 
If no arguments are provided only the row key fields, locus and alleles, are selected.

The `show` method is then used to display the selected variants.

In [18]:
row_table.select().show(5)

,
locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"


In [19]:
## the same as
mt.row_key.show(5)

,
locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"


In [20]:
### Select a field of the row table
row_table.select("qual").show(5)

,,
locus,alleles,qual
locus<GRCh37>,array<str>,float64
1:904165,"[""G"",""A""]",5.23e+04
1:909917,"[""G"",""A""]",1.58e+03
1:986963,"[""C"",""T""]",3.98e+02
1:1563691,"[""T"",""G""]",1.09e+03
1:1707740,"[""T"",""G""]",9.35e+04


In [21]:
### Select a field of the row table
row_table.select(row_table.qual, row_table.info.AC).show(5)

,,,
locus,alleles,qual,AC
locus<GRCh37>,array<str>,float64,array<int32>
1:904165,"[""G"",""A""]",5.23e+04,[518]
1:909917,"[""G"",""A""]",1.58e+03,[18]
1:986963,"[""C"",""T""]",3.98e+02,[5]
1:1563691,"[""T"",""G""]",1.09e+03,[64]
1:1707740,"[""T"",""G""]",9.35e+04,[997]


In [22]:
## The matrix table has also the colum dimension with the sample IDS. 
## To peek at the first 5 sample IDs
## s is the ID field

mt.s.show(5)

""
s
str
"""HG00096"""
"""HG00099"""
"""HG00105"""
"""HG00118"""
"""HG00129"""


#### Show attributes of entry fields. An example with the genotype field

In [23]:
## Attributes of entry fields
entry_structure = mt.entry

# To show all entry field names
print (list(entry_structure))

['GT', 'AD', 'DP', 'GQ', 'PL']


To look at the first few genotype calls, we can use entries along with select and take. The **`take` method collects the first n rows into a Python list**. Alternatively, we can use the `show` method, which prints the first n rows to the console in a table format.

In [24]:
entry_structure.take(10)

[Struct(GT=Call(alleles=[0, 0], phased=False), AD=[4, 0], DP=4, GQ=12, PL=[0, 12, 147]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[8, 0], DP=8, GQ=24, PL=[0, 24, 315]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[8, 0], DP=8, GQ=23, PL=[0, 23, 230]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[7, 0], DP=7, GQ=21, PL=[0, 21, 270]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[5, 0], DP=5, GQ=15, PL=[0, 15, 205]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[4, 0], DP=4, GQ=11, PL=[0, 11, 88]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[2, 0], DP=2, GQ=6, PL=[0, 6, 58]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[5, 0], DP=5, GQ=14, PL=[0, 14, 138]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[5, 0], DP=5, GQ=15, PL=[0, 15, 189]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[13, 0], DP=13, GQ=39, PL=[0, 39, 405])]

In [25]:
gt_expr = mt.GT # Takes the GT entry field for all samples 
gt_expr.phased.show() # Show the phased attribute of the GT field (It is False for not phased haplotypes)

,,,,
,,'HG00096','HG00099','HG00105'
locus,alleles,<expr>,<expr>,<expr>
locus<GRCh37>,array<str>,bool,bool,bool
1:904165,"[""G"",""A""]",False,False,False
1:909917,"[""G"",""A""]",False,False,False
1:986963,"[""C"",""T""]",False,False,False
1:1563691,"[""T"",""G""]",NA,False,False
1:1707740,"[""T"",""G""]",False,False,False
1:2252970,"[""C"",""T""]",False,NA,False
1:2284195,"[""T"",""C""]",False,False,False


In [26]:
gt_expr.ploidy.show()

,,,,
,,'HG00096','HG00099','HG00105'
locus,alleles,<expr>,<expr>,<expr>
locus<GRCh37>,array<str>,int32,int32,int32
1:904165,"[""G"",""A""]",2,2,2
1:909917,"[""G"",""A""]",2,2,2
1:986963,"[""C"",""T""]",2,2,2
1:1563691,"[""T"",""G""]",NA,2,2
1:1707740,"[""T"",""G""]",2,2,2
1:2252970,"[""C"",""T""]",2,NA,2
1:2284195,"[""T"",""C""]",2,2,2


In [27]:
gt_expr.summarize()

Non-missing,3048271 (98.66%)
Missing,41365 (1.34%)
Homozygous Reference,1730651
Heterozygous,741176
Homozygous Variant,576444
Ploidy,{2: 3048271}
Phased,{False: 3048271}


### Adding column fields to the matrix table starting from a metadata Table (`annotate_cols` method)

Metadata of samples (like phenotypes or geographical origin) are saved in a separate txt file imported to Hail as a Table.

A Hail MatrixTable can have any number of row fields and column fields for storing data associated with each row and column. Annotations are usually a critical part of any genetic study. **Column fields are where you’ll store information about sample phenotypes, ancestry, sex, and covariates. Row fields can be used to store information like gene membership and functional impact for use in QC or analysis**.

We'll take the text file and use it to annotate the columns in the MatrixTable.

The annotation file contains the sample ID, the population and “super-population” designations, the sample sex, and two simulated phenotypes (one binary, one discrete).

This file can be imported into Hail with `import_table`. This function produces a **Table object**. This object functions similarly to a Pandas or R DataFrame but is not constrained by the memory of a single machine and it is distributed across Spark. **The Table object, like the matrix table object, is immutable**. To interact with it locally as a Python datastructure, you should use the `take` method or transform to a Pandas dataframe (using `to_pandas` method).

**Table can have global field and row fields**. Global field is common for each element of the table whereas row fields are specific for each row. In this case, each row refers specific attributes of a sample. For example, its ID is specified in the "Sample" row field whereas the other fields show its metadata. 

In [28]:
annotation_table = (hl.import_table(annotations_fn, impute=True)
         .key_by('Sample'))

In [29]:
annotation_table.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'Sample': str 
    'Population': str 
    'SuperPopulation': str 
    'isFemale': bool 
    'PurpleHair': bool 
    'CaffeineConsumption': int32 
----------------------------------------
Key: ['Sample']
----------------------------------------


In [30]:
annotation_table.show(5)

,,,,,
Sample,Population,SuperPopulation,isFemale,PurpleHair,CaffeineConsumption
str,str,str,bool,bool,int32
"""HG00096""","""GBR""","""EUR""",False,False,4
"""HG00097""","""GBR""","""EUR""",True,True,4
"""HG00098""","""GBR""","""EUR""",False,False,5
"""HG00099""","""GBR""","""EUR""",True,False,4
"""HG00100""","""GBR""","""EUR""",True,False,5


#### Query functions and the [Hail Expression](https://hail.is/docs/0.2/overview/expressions.html) Language

Hail has a number of useful query functions that can be used for gathering statistics on our dataset. These query functions take Hail Expressions as arguments. Hail’s expressions are lazy representations of data (each data type in Hail has its own Expression class. For example, an Int32Expression represents a 32-bit integer, and a BooleanExpression represents a boolean value of True or False).

We will start by looking at some statistics of the information in our table. The [`aggregate`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.aggregate) method can be used to aggregate over rows of the table (see [Aggregation](https://hail.is/docs/0.2/guides/agg.html) and [Aggretators](https://hail.is/docs/0.2/aggregators.html#sec-aggregators) for details).

[`counter`](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.counter) is an aggregation function that counts the number of occurrences of each unique element. We can use this to pull out the population distribution by passing in a Hail Expression for the field that we want to count by.

The aggregate method is then used to aggregate something in the table across different rows and the aggregate function like counter, stats, etc... are used to specify how and what to aggregate.

In [31]:
## Population distribution
## Here counter counts unique geographycal origin label

aggregate_expression = hl.agg.counter(annotation_table.SuperPopulation)
pprint(annotation_table.aggregate(aggregate_expression))

{'AFR': 1018, 'AMR': 535, 'EAS': 617, 'EUR': 669, 'SAS': 661}


[`stats`](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.stats) is an aggregation function that produces some useful statistics about numeric collections. We can use this to see the distribution of the CaffeineConsumption phenotype.

In [32]:
## Stats perform some statistics on the specified field
## Here take stats of the caffeine consumption

aggregate_expression = hl.agg.stats(annotation_table.CaffeineConsumption)
pprint(annotation_table.aggregate(aggregate_expression))

Struct(mean=3.9837142857142855,
       stdev=1.7021055628070711,
       min=-1.0,
       max=10.0,
       n=3500,
       sum=13943.0)


The functionality demonstrated in the last few cells isn’t anything especially new: **it’s certainly not difficult to ask these questions with Pandas or R dataframes, or even Unix tools like awk. But Hail can use the same interfaces and query language to analyze collections that are much larger, like the set of variants.**

#### Grouping function to get some information within superpoulation
The Table [`group_by`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.group_by) method can be used to apply aggregation functions to different groups.

In [33]:
grp = annotation_table.group_by('SuperPopulation')

In [34]:
grp.aggregate(cnt=hl.agg.counter(annotation_table.isFemale)).show()

,
SuperPopulation,cnt
str,"dict<bool, int64>"
"""AFR""","{False:518,True:500}"
"""AMR""","{False:242,True:293}"
"""EAS""","{False:312,True:305}"
"""EUR""","{False:317,True:352}"
"""SAS""","{False:351,True:310}"


In [35]:
grp.aggregate(stats=hl.agg.stats(annotation_table.CaffeineConsumption)).show()

+-----------------+------------+-------------+-----------+-----------+---------+
| SuperPopulation | stats.mean | stats.stdev | stats.min | stats.max | stats.n |
+-----------------+------------+-------------+-----------+-----------+---------+
| str             |    float64 |     float64 |   float64 |   float64 |   int64 |
+-----------------+------------+-------------+-----------+-----------+---------+
| "AFR"           |   3.88e+00 |    1.73e+00 |  0.00e+00 |  1.00e+01 |    1018 |
| "AMR"           |   4.12e+00 |    1.66e+00 |  0.00e+00 |  9.00e+00 |     535 |
| "EAS"           |   4.01e+00 |    1.76e+00 | -1.00e+00 |  9.00e+00 |     617 |
| "EUR"           |   3.96e+00 |    1.64e+00 |  0.00e+00 |  9.00e+00 |     669 |
| "SAS"           |   4.03e+00 |    1.70e+00 |  0.00e+00 |  8.00e+00 |     661 |
+-----------------+------------+-------------+-----------+-----------+---------+

+-----------+
| stats.sum |
+-----------+
|   float64 |
+-----------+
|  3.95e+03 |
|  2.21e+03 |
|  2.47e+03 |
|  2.65e+03 |
|  2.67e+03 |
+-----------+

#### Join sample annotations with the matrix table

Using the [`annotate_cols`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_cols) method is possible to join the annotation table with the MatrixTable containing our dataset.
First, we’ll print the existing column schema using `col`. [`col`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.col) is an attribute of the matrix table that return struct expression of all column-indexed fields, including keys.
It is different from the [`cols()`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.cols) method that returns a table with all column fields in the matrix.

In [36]:
#Column table before adding per sample annotation:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str
    }
--------------------------------------------------------
Source:
Index:
    ['column']
--------------------------------------------------------


In [37]:
mt = mt.annotate_cols(pheno = annotation_table[mt.s])

# After the annotation the columns has a new field pheno,
# a struct that contains sample metadata
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }
    }
--------------------------------------------------------
Source:
Index:
    ['column']
--------------------------------------------------------


Each column "name" now specifies the sample ID and its phenotype.  

In [38]:
print(f"Metadata table samples: {annotation_table.count()}")
print(f"Matrix table samples: {mt.cols().count()}")

Metadata table samples: 3500
Matrix table samples: 284


Since there are fewer samples in our dataset than in the full thousand genomes cohort, we need to look at annotations on the dataset. We can use [`aggregate_cols`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.aggregate_cols) to get the metrics for only the samples in our dataset.

In [39]:
mt.aggregate_cols(hl.agg.counter(mt.pheno.SuperPopulation))

{'AFR': 76, 'AMR': 34, 'EAS': 72, 'EUR': 47, 'SAS': 55}

In [40]:
pprint(mt.aggregate_cols(hl.agg.stats(mt.pheno.CaffeineConsumption)))

Struct(mean=4.415492957746479,
       stdev=1.577763427465917,
       min=0.0,
       max=9.0,
       n=284,
       sum=1254.0)


The functionality demonstrated in the last few cells isn’t anything especially new: it’s certainly not difficult to ask these questions with **Pandas** or **R** dataframes, or even Unix tools like `awk`. But **Hail** can use the same interfaces and query language to analyze collections that are much larger, like the set of variants.

Here we calculate the counts of each of the 12 possible unique SNPs (4 choices for the reference base * 3 choices for the alternate base).

To do this, we need to get the alternate allele of each variant and then count the occurences of each unique ref/alt pair. This can be done with Hail’s **counter function**.

In [41]:
struct_expr = hl.Struct(ref=mt.alleles[0], alt=mt.alleles[1])
snp_counts = mt.aggregate_rows(hl.agg.counter(struct_expr))
pprint(snp_counts)

# We can list the counts in descending order using Python’s Counter class.
from collections import Counter
counts = Counter(snp_counts)
counts.most_common()

{Struct(ref='T', alt='C'): 1864,
 Struct(ref='G', alt='A'): 2367,
 Struct(ref='C', alt='A'): 494,
 Struct(ref='A', alt='G'): 1929,
 Struct(ref='A', alt='T'): 75,
 Struct(ref='C', alt='T'): 2418,
 Struct(ref='T', alt='G'): 466,
 Struct(ref='G', alt='C'): 111,
 Struct(ref='A', alt='C'): 451,
 Struct(ref='T', alt='A'): 77,
 Struct(ref='C', alt='G'): 150,
 Struct(ref='G', alt='T'): 477}


[(Struct(ref='C', alt='T'), 2418),
 (Struct(ref='G', alt='A'), 2367),
 (Struct(ref='A', alt='G'), 1929),
 (Struct(ref='T', alt='C'), 1864),
 (Struct(ref='C', alt='A'), 494),
 (Struct(ref='G', alt='T'), 477),
 (Struct(ref='T', alt='G'), 466),
 (Struct(ref='A', alt='C'), 451),
 (Struct(ref='C', alt='G'), 150),
 (Struct(ref='G', alt='C'), 111),
 (Struct(ref='T', alt='A'), 77),
 (Struct(ref='A', alt='T'), 75)]

**The same Python, R, and Unix tools could do this work as well, but we’re starting to hit a wall - the latest gnomAD release publishes about 250 million variants, and that won’t fit in memory on a single computer.**

What about genotypes? Hail can query the collection of all genotypes in the dataset, and this is getting large even for our tiny dataset. Our 284 samples and 10,000 variants produce 10 million unique genotypes. The gnomAD dataset has about 5 trillion unique genotypes.

#### Hail plotting functions
Hail plotting functions allow Hail fields as arguments, so we can pass in the DP field directly here. If the range and bins arguments are not set, this function will compute the range based on minimum and maximum values of the field and use the default 50 bins.

In [42]:
p = hl.plot.histogram(mt.DP, range=(0,30), bins=30, title='DP Histogram', legend='DP')
show(p)

### Quality control

QC is where analysts spend most of their time with sequencing datasets. QC is an iterative process, and is different for every project: there is no “push-button” solution for QC. Each time the Broad collects a new group of samples, it finds new batch effects. However, by practicing open science and discussing the QC process and decisions with others, we can establish a set of best practices as a community.

QC is entirely based on the ability to understand the properties of a dataset. Hail attempts to make this easier by providing the [`sample_qc`](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc) function, which produces a set of useful metrics and stores them in a column field.

In [43]:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }
    }
--------------------------------------------------------
Source:
Index:
    ['column']
--------------------------------------------------------


In [44]:
# sample_qc is a hail genetic method to compute per-sample metrics useful for quality control.
mt = hl.sample_qc(mt)
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }, 
        sample_qc: struct {
            dp_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            gq_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            call_rate: float64, 
            n_called: int64, 
            n_not_called: int64, 
            n_filtered: int64, 
            n_hom_ref: int64, 
            n_het: int64, 
            n_hom_var: int64, 
            n_non_ref: int64, 
            n_singleton: int64, 
            n_snp: int64, 
            n_insertio

In [45]:
##Plotting the QC metrics is a good place to start.
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(p)

In [46]:
p = hl.plot.histogram(mt.sample_qc.n_not_called, legend='Not called')
show(p)

In [47]:
## Checking corralations between the mean value of dp and the call rate
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
p.line([2,22], [0.97,0.97], color='red', line_width=2)
p.line([4,4], [0.878,1.0], color='red', line_width=2)
show(p)

#### Removing outliers

Removing outliers from the dataset will generally improve association results. We can make arbitrary cutoffs and use them to filter.
Using matrix table [`filter_cols`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.filter_cols) method it is possible to **create a new matrix table considering samples with the DP mean >= 4 and a call rate >= 0.97**. Samples that don't satisfy these criteria are removed.
The filtering method does not perform in-place filtering, so the result must be assigned to a variable for the changes to take effect.

In [48]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
print('After filter, %d/284 samples remain.' % mt.count_cols())

After filter, 250/284 samples remain.


In [49]:
mt.GT.summarize()

Non-missing,2699274 (99.25%)
Missing,20476 (0.75%)
Homozygous Reference,1524538
Heterozygous,672812
Homozygous Variant,501924
Ploidy,{2: 2699274}
Phased,{False: 2699274}


Next is genotype QC. It’s a good idea to filter out genotypes where the reads aren’t where they should be: if we find a genotype called homozygous reference with >10% alternate reads, a genotype called homozygous alternate with >10% reference reads, or a genotype called heterozygote without a ref / alt balance near 1:1, it is likely to be an error.

In a low-depth dataset like 1KG, it is hard to detect bad genotypes using this metric, since a read ratio of 1 alt to 10 reference can easily be explained by binomial sampling. However, in a high-depth dataset, a read ratio of 10:100 is a sure cause for concern!



In [50]:
ab = mt.AD[1] / hl.sum(mt.AD)

filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))

fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
mt = mt.filter_entries(filter_condition_ab)

Filtering 3.60% entries out of downstream analysis.


Variant QC computes per per-variant metric useful for quality control. It is a bit more of the same of sample_qc: we can use the [`variant_qc`](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc) function to produce a variety of useful statistics, plot them, and filter. This is made at row level beacause they are stats on variants.

In [51]:
mt = hl.variant_qc(mt)
mt.row.describe()

--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        rsid: str, 
        qual: float64, 
        filters: set<str>, 
        info: struct {
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            BaseQRankSum: float64, 
            ClippingRankSum: float64, 
            DP: int32, 
            DS: bool, 
            FS: float64, 
            HaplotypeScore: float64, 
            InbreedingCoeff: float64, 
            MLEAC: array<int32>, 
            MLEAF: array<float64>, 
            MQ: float64, 
            MQ0: int32, 
            MQRankSum: float64, 
            QD: float64, 
            ReadPosRankSum: float64, 
            set: str
        }, 
        variant_qc: struct {
            dp_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 

In [52]:
p = hl.plot.histogram(mt.variant_qc.call_rate, legend='Variant QC call rate')
show(p)

In [53]:
p = hl.plot.histogram(mt.variant_qc.dp_stats.mean, legend='Variant QC DP')
show(p)

## Let’s do a GWAS!

First, we need to restrict to variants that are :

- common (we’ll use a cutoff of 1%)
- not so far from Hardy-Weinberg equilibrium as to suggest sequencing error

In [54]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01) # It takes variants for which the alternate allele has a frequency larger than 1%
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6) # Hardy-Weinberg equilibrium pvalue cut-off
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

Samples: 250  Variants: 9095
Samples: 250  Variants: 7774


These filters removed about 15% of sites (we started with a bit over 10,000). This is NOT representative of most sequencing datasets! We have already downsampled the full thousand genomes dataset to include more common variants than we’d expect by chance.

In Hail, the association tests accept column fields for the sample phenotype and covariates. Since we’ve already got our phenotype of interest (caffeine consumption) in the dataset, we are good to go:

In [55]:
gwas = hl.linear_regression_rows(y=mt.pheno.CaffeineConsumption,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0])
gwas.row.describe()

[Stage 78:>                                                         (0 + 1) / 1]

--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        n: int32, 
        sum_x: float64, 
        y_transpose_x: float64, 
        beta: float64, 
        standard_error: float64, 
        t_stat: float64, 
        p_value: float64
    }
--------------------------------------------------------
Source:
Index:
    ['row']
--------------------------------------------------------


Looking at the bottom of the above printout, you can see the linear regression adds new row fields for the beta, standard error, t-statistic, and p-value.

Hail makes it easy to visualize results! Let’s make a Manhattan plot:

In [56]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

This doesn’t look like much of a skyline. Let’s check whether our GWAS was well controlled using a Q-Q (quantile-quantile) plot.

In [57]:
p = hl.plot.qq(gwas.p_value)
show(p)

## Confounded!

The observed p-values drift away from the expectation immediately. Either every SNP in our dataset is causally linked to caffeine consumption (unlikely), or there’s a confounder.

We didn’t tell you, but sample ancestry was actually used to simulate this phenotype. This leads to a **stratified distribution** of the phenotype. The solution is to include ancestry as a covariate in our regression.

The **linear_regression_rows** function can also take column fields to use as covariates. We already annotated our samples with reported ancestry, but it is good to be skeptical of these labels due to human error. Genomes don’t have that problem! Instead of using reported ancestry, we will use genetic ancestry by including computed principal components in our model.

The **pca** function produces eigenvalues as a list and sample PCs as a Table, and can also produce variant loadings when asked. The **hwe_normalized_pca** function does the same, using HWE-normalized genotypes for the PCA.

In [58]:
mt.GT.show()

,,,,,
,,'HG00096','HG00099','HG00105','HG00118'
locus,alleles,GT,GT,GT,GT
locus<GRCh37>,array<str>,call,call,call,call
1:904165,"[""G"",""A""]",0/0,0/0,0/0,0/0
1:1563691,"[""T"",""G""]",NA,0/0,0/0,0/0
1:1707740,"[""T"",""G""]",0/1,0/1,0/1,0/0
1:2284195,"[""T"",""C""]",1/1,0/1,0/1,0/1
1:2779043,"[""T"",""C""]",0/1,0/1,1/1,0/0
1:2944527,"[""G"",""A""]",0/0,0/1,NA,0/1
1:3761547,"[""C"",""A""]",0/0,0/0,0/0,0/0


In [59]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

In [60]:
pprint(eigenvalues)

[18.084111467840707,
 9.984076405601847,
 3.540687229805949,
 2.655598108390125,
 1.596852701724399,
 1.5405241027955296,
 1.507713504116216,
 1.4744976712480349,
 1.467690539034742,
 1.4461994473306554]


In [61]:
pcs.show(5, width=100)

,
s,scores
str,array<float64>
"""HG00096""","[1.22e-01,2.81e-01,-1.10e-01,-1.27e-01,6.68e-02,3.29e-03,-2.26e-02,4.26e-02,-9.30e-02,1.83e-01]"
"""HG00099""","[1.14e-01,2.89e-01,-1.06e-01,-6.78e-02,4.72e-02,2.87e-02,5.28e-03,-1.57e-02,1.75e-02,-1.98e-02]"
"""HG00105""","[1.09e-01,2.79e-01,-9.95e-02,-1.06e-01,8.79e-02,1.44e-02,2.80e-02,-3.38e-02,-1.08e-03,2.25e-02]"
"""HG00118""","[1.26e-01,2.95e-01,-7.58e-02,-1.08e-01,1.76e-02,7.91e-03,-5.25e-02,3.05e-02,2.00e-02,-7.78e-02]"
"""HG00129""","[1.06e-01,2.86e-01,-9.69e-02,-1.15e-01,1.03e-02,2.65e-02,-8.51e-02,2.49e-02,5.67e-02,-8.31e-03]"


Now that we’ve got principal components per sample, we may as well plot them! Human history exerts a strong effect in genetic datasets. Even with a 50MB sequencing dataset, we can recover the major human populations.

In [62]:
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

In [63]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'pheno': struct {
        Population: str, 
        SuperPopulation: str, 
        isFemale: bool, 
        PurpleHair: bool, 
        CaffeineConsumption: int32
    }
    'sample_qc': struct {
        dp_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64, 
        n_hom_ref: int64, 
        n_het: int64, 
        n_hom_var: int64, 
        n_non_ref: int64, 
        n_singleton: int64, 
        n_snp: int64, 
        n_insertion: int64, 
        n_deletion: int64, 
        n_transition: int64, 
        n_transv

In [64]:
p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=mt.pheno.SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

Now we can rerun our linear regression, controlling for sample sex and the first few principal components. We’ll do this with input variable the number of alternate alleles as before, and again with input variable the genotype dosage derived from the PL field.

In [65]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])


In [66]:
gwas.show()

,,,,,,,,
locus,alleles,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value
locus<GRCh37>,array<str>,int32,float64,float64,float64,float64,float64,float64
1:904165,"[""G"",""A""]",250,5.77e+01,2.48e+02,-6.10e-02,1.94e-01,-3.14e-01,7.54e-01
1:1563691,"[""T"",""G""]",250,1.10e+01,5.38e+01,4.60e-01,4.13e-01,1.11e+00,2.66e-01
1:1707740,"[""T"",""G""]",250,8.43e+01,3.77e+02,8.29e-02,1.70e-01,4.87e-01,6.26e-01
1:2284195,"[""T"",""C""]",250,1.49e+02,6.83e+02,-1.16e-01,1.42e-01,-8.19e-01,4.13e-01
1:2779043,"[""T"",""C""]",250,3.73e+02,1.67e+03,3.13e-01,1.50e-01,2.09e+00,3.78e-02
1:2944527,"[""G"",""A""]",250,1.01e+02,4.62e+02,-9.24e-02,1.73e-01,-5.34e-01,5.94e-01
1:3761547,"[""C"",""A""]",250,5.02e+00,1.51e+01,-6.26e-01,6.76e-01,-9.26e-01,3.55e-01
1:3803755,"[""T"",""C""]",250,3.57e+02,1.58e+03,-2.06e-02,1.34e-01,-1.53e-01,8.78e-01


We’ll first make a Q-Q plot to assess inflation…

In [67]:
p = hl.plot.qq(gwas.p_value)
show(p)

That’s more like it! This shape is indicative of a well-controlled (but not especially well-powered) study. And now for the Manhattan plot:

In [68]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

We have found a caffeine consumption locus!

#### How to save the Matrix table

In [69]:
mt_out_fn = 's3://data-hail/1kg_after_gwas.mt'
mt.write(mt_out_fn, overwrite=True)